In [32]:
import numpy as np
import cv2
import glob
import math

In [33]:
def ShowResizedIm(img,windowname,scale):
    """
    opencv imshow resized image on a new window
    Parameters:
        -img: image
        -window: window name
        -scale: size of the display image will be divided by this value(ex. scale=2 will make image 2 time smaller)
    """
    cv2.namedWindow(windowname, cv2.WINDOW_NORMAL)        # Create window with freedom of dimensions
    height, width = img.shape[:2]   #get image dimension
    cv2.resizeWindow(windowname,int(width/scale) ,int(height/scale))                    # Resize image
    cv2.imshow(windowname, img)                            # Show image

In [34]:
def resizeHalf(img):
    """
    opencv resized image in half
    Parameters:
        -img: image
    Returns:
        -resized: resized image
    """
    width = int(img.shape[1]/2)
    height = int(img.shape[0]/2)
    dim = (width, height)
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return resized

In [35]:
def findLineCoordinate(cY, cX, canv_H, canv_W,theta):
    """
    Find coordinates of both ends of a line from theta and a single point

    Parameters:
        -cY: an int of vertical coordinate of the single point
        -cX: an int of horizontal coordinate of the single point
        -canv_H: an int of hight of a canvas
        -canv_W: an int of width of a canvas
        -theta: a size of angle in degree measured form right-handed horizontal line of the point 

    Returns:
        -v1,u1,v2,u2: coordinate of both end of the line(v is vertical, u is horizontal)
    """
    if theta < 90:
        #find first coordinate of the line
        u1 = int(cY-(math.tan(math.radians(theta))*(canv_W-cX)))
        #clip of some part of the line that extend out of the image
        if u1 < 0:
            v1 = int(canv_W-1+(u1/math.tan(math.radians(theta))))
            u1 = 0
        else:
            v1 = canv_W-1
        #find second coordinate of the line
        u2 = int(cY+(math.tan(math.radians(theta))*cX))
        #clip of some part of the line that extend out of the image
        if u2 > canv_H-1:
            v2 = int((u2-canv_H-1)/math.tan(math.radians(theta)))
            u2 = canv_H-1
        else:
            v2 = 0
    elif theta == 90:
        #find first coordinate of the line
        u1 = 0
        v1 = cX
        #find second coordinate of the line
        u2 = canv_H
        v2 = cX
    else:
        #find first coordinate of the line
        u1 = int(cY+(math.tan(math.radians(theta))*(canv_W-1-cX)*(-1)))
        #clip of some part of the line that extend out of the image
        if u1 < 0:
            v1 = int(u1/math.tan(math.radians(theta)))
            u1 = 0
        else:
            v1 = canv_W-1
        #find second coordinate of the line
        u2 = int(cY-(math.tan(math.radians(theta))*cX*(-1)))
        #clip of some part of the line that extend out of the image
        if u2 > canv_H-1:
            v2 = int(canv_W-1+(u2-canv_H-1)/math.tan(math.radians(theta)))
            u2 = canv_H-1
        else:
            v2 = 0
    return v1,u1,v2,u2

In [36]:
def createLineIterator(P1, P2, img):
    """
    Produces and array that consists of the coordinates and intensities of each pixel in a line between two points

    Parameters:
        -P1: a numpy array that consists of the coordinate of the first point (x,y)
        -P2: a numpy array that consists of the coordinate of the second point (x,y)
        -img: the image being processed

    Returns:
        -it: a numpy array that consists of the coordinates and intensities of each pixel in the radii (shape: [numPixels, 3], row = [x,y,intensity])     
    """
    #define local variables for readability
    imageH = img.shape[0]
    imageW = img.shape[1]
    P1X = P1[0]
    P1Y = P1[1]
    P2X = P2[0]
    P2Y = P2[1]

    #difference and absolute difference between points
    #used to calculate slope and relative location between points
    dX = P2X - P1X
    dY = P2Y - P1Y
    dXa = np.abs(dX)
    dYa = np.abs(dY)

    #predefine numpy array for output based on distance between points
    itbuffer = np.empty(shape=(np.maximum(int(dYa),int(dXa)),3),dtype=np.float32)
    itbuffer.fill(np.nan)

    #Obtain coordinates along the line using a form of Bresenham's algorithm
    negY = P1Y > P2Y
    negX = P1X > P2X
    if P1X == P2X: #vertical line segment
        itbuffer[:,0] = P1X
        if negY:
            itbuffer[:,1] = np.arange(P1Y - 1,P1Y - dYa - 1,-1)
        else:
            itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)              
    elif P1Y == P2Y: #horizontal line segment
        itbuffer[:,1] = P1Y
        if negX:
            itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
        else:
            itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
    else: #diagonal line segment
        steepSlope = dYa > dXa
        if steepSlope:
            slope = dX.astype(np.float32)/dY.astype(np.float32)
            if negY:
                itbuffer[:,1] = np.arange(P1Y-1,P1Y-dYa-1,-1)
            else:
                itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)
            itbuffer[:,0] = (slope*(itbuffer[:,1]-P1Y)).astype(np.int) + P1X
        else:
            slope = dY.astype(np.float32)/dX.astype(np.float32)
            if negX:
                itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
            else:
                itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
            itbuffer[:,1] = (slope*(itbuffer[:,0]-P1X)).astype(np.int) + P1Y

    #Remove points outside of image
    colX = itbuffer[:,0]
    colY = itbuffer[:,1]
    itbuffer = itbuffer[(colX >= 0) & (colY >=0) & (colX<imageW) & (colY<imageH)]

    #Get intensities from img ndarray
    itbuffer[:,2] = img[itbuffer[:,1].astype(np.uint),itbuffer[:,0].astype(np.uint)]

    return itbuffer

In [37]:
#http://www.gregbugaj.com/?p=524
class Point(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def euclidean_distance(self, other):
        p1 = (other.x - self.x) ** 2
        p2 = (other.y - self.y) ** 2
        vd = (p1 + p2) ** 0.5
        return vd


class PointList(list):
    fraction = .7

    def hausdorffPHD(self, other):
        max_distance = 0
        ranking = []
        for i, a in enumerate(self):
            min_distance = float('inf')

            for j, b in enumerate(other):
                ed = a.euclidean_distance(b)

                if ed < min_distance:
                    min_distance = ed

            ranking.append(min_distance)
        ranking = sorted(ranking)
        k = int(len(self) * self.fraction)-1
        return ranking[k]


def hausdorff(seta, setb):
    habPHD = seta.hausdorffPHD(setb)
    hbaPHD = setb.hausdorffPHD(seta)
    distancePHD = max(habPHD, hbaPHD)
    #print(f'hd = {distancePHD:.4f}\t {habPHD:.4f}\t {hbaPHD:.4f}')
    return distancePHD



In [38]:
#find Vessel symmetry line degree
def findVSL(edges,K = 3,degree_step = 5,showVSL = 1):
    """
    Find vessel symmetry line by creating symmetry line and find symetry of pixel in the image 
    with Partial Hausdorff distance. The line will rotate 5 degree and the symmetry finding will 
    be done until rotate angle reach 180 degree. Line that has the most symmetry score will be 
    symmetry line of the image 
    Parameters:
        -edges: binary edges image
        -K: amount of output line
        -degree_step: degree angle for each rotation
        -showVSL: show VSL value every 5 degree rotation (set to 1 to show the value)
    Returns:
        -canvas: binary image of the symmetry lines on the edges image
        -VSL_list: list of the angle degree of the output symetry lines (the degree is measured 
        from right side horizontal line)
    """
    #calculate center of mass
    M = cv2.moments(edges)
    # calculate x,y coordinate of center
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    #calculate image size
    img_H,img_W = edges.shape[:2]
    #create mask to preview spining line
    mask = np.zeros((img_H,img_W), dtype=np.uint8)#test
    #Store VSL value for each line
    VSL = {}
    for theta in range(0,180,degree_step):
        #find line coordinate from theta
        mask_line_v1,mask_line_u1,mask_line_v2,mask_line_u2 = findLineCoordinate(cY, cX, img_H, img_W,theta)
        cv2.line(mask,(mask_line_v1,mask_line_u1),(mask_line_v2,mask_line_u2),(255,255,255),1)#test
        #find all pixels on the line
        line = createLineIterator(np.array((mask_line_v1,mask_line_u1)),np.array((mask_line_v2,mask_line_u2)), mask)
        #create mask to preview subline on the line
        submask = np.zeros((img_H,img_W), dtype=np.uint8)
        #find new theta for orthogonal subline which is in range 0-179
        if theta < 90:
            theta2 = theta+90
        else:  
            theta2 = theta-90
        #Store PHD output for each subline
        PHD_val = []
        mask_test = np.zeros((img_H,img_W), dtype=np.uint8)#test
        cv2.line(mask_test,(mask_line_v1,mask_line_u1),(mask_line_v2,mask_line_u2),(255,255,255),1)#test
        for pixel in line:
            #pixel[0] is x, pixel[1] is y
            sub_line_v1,sub_line_u1,sub_line_v2,sub_line_u2 = findLineCoordinate(pixel[1], pixel[0], img_H, img_W,theta2)
            cv2.line(submask,(sub_line_v1,sub_line_u1),(sub_line_v2,sub_line_u2),(255,255,255),1)      
            #crate set1 pm
            set1 = PointList()
            #crate set2 pm
            set2 = PointList()
            #crate list of pixels on a line contaied coordinate and value
            subline1 = createLineIterator(np.array((sub_line_v1,sub_line_u1)),np.array((pixel[0],pixel[1])), edges)
            #crate list of pixels on a line contaied coordinate and value
            subline2 = createLineIterator(np.array((sub_line_v2,sub_line_u2)),np.array((pixel[0],pixel[1])), edges)
            #if subline1 is not empty 
            if np.any(subline1[:,2]) and not np.any(subline2[:,2]):
                for subpixel in subline1:
                    if subpixel[2] == 255:
                        set1.append(Point(abs(pixel[0] - subpixel[0]),abs(pixel[1] - subpixel[1])))
                        set2.append(Point(abs(pixel[0] - sub_line_v2),abs(pixel[1] - sub_line_u2)))
            #if subline2 is not empty 
            if not np.any(subline1[:,2]) and np.any(subline2[:,2]):
                for subpixel in subline2:
                    if subpixel[2] == 255:
                        set1.append(Point(abs(pixel[0] - sub_line_v1),abs(pixel[1] - sub_line_u1)))
                        set2.append(Point(abs(pixel[0] - subpixel[0]),abs(pixel[1] - subpixel[1])))
            #run through all pixels on sublines if both lines are not empty
            if np.any(subline1[:,2]) and np.any(subline2[:,2]):
                for subpixel in subline1:
                    if subpixel[2] == 255:
                        #add vector of subpixel to set1
                        set1.append(Point(abs(pixel[0] - subpixel[0]),abs(pixel[1] - subpixel[1])))
                        mask_test[int(subpixel[1])][int(subpixel[0])] = 255#test
                for subpixel in subline2:
                    if subpixel[2] == 255:
                        #add vector of subpixel to set2
                        set2.append(Point(abs(pixel[0] - subpixel[0]),abs(pixel[1] - subpixel[1])))
                        mask_test[int(subpixel[1])][int(subpixel[0])] = 255#test
            #find PHD
            if len(set1) and len(set2):
                distance = hausdorff(set1, set2)
                PHD_val.append(distance)

                if theta == 1000:
                    print(distance)
                    ShowResizedIm(np.hstack([mask_test,]),"mark",2)#test
                    cv2.waitKey(0)#test
                    cv2.destroyAllWindows()#test
        #find VSL
        if len(PHD_val):
            #VSL[theta]= sum(PHD_val)/len(line)
            VSL[theta]= sum(PHD_val)/len(PHD_val)
        else:
            VSL[theta] = 0
        if showVSL:
            print(VSL[theta])
            
    
    #find list of VSL
    VSL_list = []
    VSL = sorted(VSL.items(), key = lambda kv:(kv[1], kv[0]))
    for i,element in enumerate(VSL):
        if i < K:
            VSL_list.append(element[0])
        else:
            break
    #draw lines in a canvas
    canvas = edges.copy()
    for i in VSL_list:
        VSL_v1,VSL_u1,VSL_v2,VSL_u2 = findLineCoordinate(cY, cX, img_H, img_W,i)
        cv2.line(canvas,(VSL_v1,VSL_u1),(VSL_v2,VSL_u2),(255,255,255),1)
    return canvas,VSL_list


In [191]:
def findCoordinateMaxVCC(img,edges,VSL_list,semi_annula_radii):
    """
    Find coordinate of maximum vessel component count. This function counts vessel components those overlap with
    semi annula mask at each pixel along the VSL. Pixel with highest VCC will be the output coordinate.
    Parameters:
        -img: binary erosion image
        -edges: binary edges image just to find centroid
        -VSL_list: list of degree angle of the VSL line
        -semi_annula_radii: outer radaius of the semi annula mask
    Returns:
        -xVCC,yVCC: coordinate of the maximum VCC pixel
    """
    #Create semi annula mask
    semi_annula_mask_R = np.zeros(((semi_annula_radii*2)+1,(semi_annula_radii*2)+1), dtype=np.uint8)
    cv2.circle(semi_annula_mask_R, (semi_annula_radii, semi_annula_radii),semi_annula_radii , (255, 255, 255), -1)
    cv2.circle(semi_annula_mask_R, (semi_annula_radii, semi_annula_radii),int(semi_annula_radii/2) , (0, 0, 0), -1)
    semi_annula_mask_L = semi_annula_mask_R.copy() 
    cv2.rectangle(semi_annula_mask_R, (0,0), (semi_annula_radii,semi_annula_radii*2),(0, 0, 0), -1)
    cv2.rectangle(semi_annula_mask_L, (semi_annula_radii,0), (semi_annula_radii*2,semi_annula_radii*2),(0, 0, 0), -1)
    #calculate image size
    img_H,img_W = img.shape[:2]
    #calculate center of mass
    M = cv2.moments(edges)
    # calculate x,y coordinate of center
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    
    maxVCC_list = []
    for VSL in VSL_list:
        #find coordinate of both end of the line form VSL theta degree
        line_v1,line_u1,line_v2,line_u2 = findLineCoordinate(cY, cX, img_H, img_W,VSL)
        #create list of pixel coordinate and value along the line
        psk = createLineIterator(np.array((line_v1,line_u1)),np.array((line_v2,line_u2)), img)
        #crate list of VCC that will contain vessel component count from intersection of annula mask and eroded edges image
        VCC = []
        #for each pixel along the line
        for pixel in psk:
            #crop the image with annula mask size
            imCrop = img[int(pixel[1]-semi_annula_radii-1):int(pixel[1]+semi_annula_radii),
                                        int(pixel[0]-semi_annula_radii-1):int(pixel[0]+semi_annula_radii)]
            #to prevent processing at image rim
            if imCrop.shape[:2] == semi_annula_mask_R.shape[:2]:
                bit_and_R = cv2.bitwise_and(imCrop,semi_annula_mask_R)
                __,contours_R, hierarchy_R = cv2.findContours(bit_and_R, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                bit_and_L = cv2.bitwise_and(imCrop,semi_annula_mask_L)
                __,contours_L, hierarchy_L = cv2.findContours(bit_and_L, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                VCC_R = 0
                for cnt in contours_R:
                    area = cv2.contourArea(cnt)
                    if area > 100:
                        VCC_R = VCC_R + 1
                VCC_L = 0
                for cnt in contours_L:
                    area = cv2.contourArea(cnt)
                    if area > 100:
                        VCC_L = VCC_L + 1
                VCC.append(max(VCC_R,VCC_L))
            else:
                VCC.append(0)
        maxVCC_list.append(max(VCC))
    #print(maxVCC_list)
    #print(np.argmax(maxVCC_list))
    #find coordinate of both end of the line form VSL theta degree
    line_v1,line_u1,line_v2,line_u2 = findLineCoordinate(cY, cX, img_H, img_W,VSL_list[np.argmax(maxVCC_list)])
    #create list of pixel coordinate and value along the line
    psk = createLineIterator(np.array((line_v1,line_u1)),np.array((line_v2,line_u2)), img)
    VCC = []
    #for each pixel along the line
    for pixel in psk:
        #crop the image with annula mask size
        imCrop = img[int(pixel[1]-semi_annula_radii-1):int(pixel[1]+semi_annula_radii),
                                    int(pixel[0]-semi_annula_radii-1):int(pixel[0]+semi_annula_radii)]
        #to prevent processing at image rim
        if imCrop.shape[:2] == semi_annula_mask_R.shape[:2]:
            bit_and_R = cv2.bitwise_and(imCrop,semi_annula_mask_R)
            __,contours_R, hierarchy_R = cv2.findContours(bit_and_R, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            bit_and_L = cv2.bitwise_and(imCrop,semi_annula_mask_L)
            __,contours_L, hierarchy_L = cv2.findContours(bit_and_L, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            canvas = np.zeros(bit_and_R.shape)
            VCC_R = 0
            for cnt in contours_R:
                area = cv2.contourArea(cnt)
                if area > 100:
                    VCC_R = VCC_R + 1
                    cv2.drawContours(canvas,[cnt],-1,(255,255,255),-1)
            VCC_L = 0
            for cnt in contours_L:
                area = cv2.contourArea(cnt)
                if area > 100:
                    VCC_L = VCC_L + 1
                    cv2.drawContours(canvas,[cnt],-1,(255,255,255),-1)
            VCC.append(max(VCC_R,VCC_L))
            #if max(VCC_R,VCC_L) == 11:
            #    ShowResizedIm(canvas,"windowname",1)
            #    cv2.waitKey(0)
            #    cv2.destroyAllWindows()
            #VCC.append(VCC_R+VCC_L)
        else:
            VCC.append(0)
    print(len(VCC))
    #find argmax of the list VCC
    S = np.argmax(VCC)
    print(len(psk))
    #find output coordinate
    xVCC = int(psk[S][0])
    yVCC = int(psk[S][1])
    return xVCC,yVCC

In [194]:
#VCC
xVCC,yVCC = findCoordinateMaxVCC(erosion,edges,VSL,semi_annula_radii)
previewAnnulaMask(erosion,xVCC,yVCC,semi_annula_radii)
#ODC final
x_out,y_out = findFinalODC(img,xVCC,yVCC,int(semi_annula_radii/2))
previewAnnulaMask(img,x_out,y_out,semi_annula_radii)

1263
1263


In [40]:
def previewAnnulaMask(img,x,y,semi_annula_radii):
    canvas = img.copy()
    cv2.circle(canvas , (x,y), 5, (255, 255, 255), -1)
    cv2.circle(canvas , (x,y), semi_annula_radii, (255, 255, 255), 1)
    cv2.circle(canvas , (x,y), int(semi_annula_radii/2), (255, 255, 255), 1)
    cv2.imshow("canvas",canvas)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
#previewAnnulaMask(erosion,xVCC,yVCC,semi_annula_radii)

In [41]:
def findCentroidCircle(img,x,y,radius):
    #crop image
    imCrop = img[int(y-radius-1):int(y+radius),int(x-radius-1):int(x+radius)]
    #create circle mask
    mask = np.zeros(((radius*2)+1,(radius*2)+1), dtype=np.uint8)
    cv2.circle(mask, (radius, radius),radius , (255, 255, 255), -1)
    #apply mask to crop image
    imCrop= cv2.bitwise_and(imCrop,mask)
    M = cv2.moments(imCrop)
    # calculate x,y coordinate of center
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    return cX+int(x-radius-1),cY+int(y-radius-1)
#cX2,cY2 = findCentroidCircle(erosion,psk[S][0],psk[S][1],int(semi_annula_radii/2))

In [42]:
def drawCircleImShow(img,x,y,radius,color,thickness):
    canvas = img.copy()
    cv2.circle(canvas , (x,y), radius, color, thickness)
    cv2.imshow("canvas",canvas)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
#drawCircleImShow(erosion,cX2,cY2,4,(255,255,255),-1)

In [43]:
def findFinalODC(img,x,y,radius,E = 1,iteration = 1000):
    cX2,cY2 = findCentroidCircle(img,x,y,radius)
    error = 100
    count = 0
    while error > E and count < iteration:
        cX1,cY1 = findCentroidCircle(img,cX2,cY2,radius)
        error = ((cX1-cX2)**2+(cY1-cY2)**2)**0.5
        cX2 = cX1
        cY2 = cY1
        count = count+1
        #drawCircleImShow(img,cX2,cY2,6,(255,255,255),-1)
    return cX2,cY2
#findFinalODC(erosion,psk[S][0],psk[S][1],int(semi_annula_radii/2))

In [200]:
def main():
    #=========USER START================
    #folder path
    path = 'bv image sarai/*.jpg' 
    image_number=50
    show_image_flag = 1
    semi_annula_radii = 150
    #=========USER END================
    image_list = []
    for filename in glob.glob(path):
        image_list.append(filename)
    img = cv2.imread(image_list[image_number],0)
    img = resizeHalf(img)
    print(image_list[image_number])
    kernel_erosion = np.ones((5,5),np.uint8)
    erosion = cv2.erode(img,kernel_erosion,iterations = 1)
    edges = cv2.Canny(erosion,100,200)
    canvas = edges.copy()
    # calculate moments of binary image
    M = cv2.moments(edges)
    # calculate x,y coordinate of center
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    # put text and highlight the center
    if show_image_flag:
        cv2.circle(canvas , (cX, cY), 5, (255, 255, 255), -1)
        cv2.putText(canvas , "centroid", (cX - 25, cY - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        ShowResizedIm(np.hstack([img,erosion,canvas,]),"mark",2)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    #VSL
    VSL_img,VSL = findVSL(edges,showVSL = 1)
    if show_image_flag:
        ShowResizedIm(np.hstack([VSL_img,]),"mark",1)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    #VCC
    xVCC,yVCC = findCoordinateMaxVCC(erosion,edges,VSL,semi_annula_radii)
    if show_image_flag:
        previewAnnulaMask(img,xVCC,yVCC,semi_annula_radii)
    #ODC final
    x_out,y_out = findFinalODC(erosion,xVCC,yVCC,int(semi_annula_radii/2))
    if show_image_flag:
        previewAnnulaMask(erosion,x_out,y_out,semi_annula_radii)

In [201]:
%%time
if __name__ == '__main__':
    main()

bv image sarai\267_089.jpg
74.46943483275663
174.47742499541135
264.6950931407126
129.271699732792
98.23087982451233
113.65258411517824
188.51857071866561
128.8788435708456
99.77802182644568
101.52392177823766
130.6644439314218
119.61275356583988
101.7830848908183
102.16351209577186
93.80202962789367
104.76435088504496
120.96309542624473
113.80960180833853
98.83673469387755
120.60523341695941
114.83855290693418
112.25446796634782
100.06934701325007
90.56495741931093
99.87319834556418
85.0243742429932
96.86127375255101
100.06437806388105
87.68117581393267
78.63490487250121
77.71834544785058
85.28036410169467
84.91951637704287
92.29675589315879
85.4088634574633
79.93567691731687
1263
1263
Wall time: 2min 20s


In [130]:
#=========USER START================
#folder path
path = 'bv image sarai/*.jpg' 
image_number=0
semi_annula_radii = 100
#=========USER END================
image_list = []
for filename in glob.glob(path):
    image_list.append(filename)
img = cv2.imread(image_list[image_number],0)
img = resizeHalf(img)
print(image_list[image_number])

bv image sarai\267_001.jpg


In [131]:
kernel_erosion = np.ones((5,5),np.uint8)
erosion = cv2.erode(img,kernel_erosion,iterations = 1)
edges = cv2.Canny(erosion,100,200)
canvas = edges.copy()
# calculate moments of binary image
M = cv2.moments(edges)
# calculate x,y coordinate of center
cX = int(M["m10"] / M["m00"])
cY = int(M["m01"] / M["m00"])
# put text and highlight the center
cv2.circle(canvas , (cX, cY), 5, (255, 255, 255), -1)
cv2.putText(canvas , "centroid", (cX - 25, cY - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
ShowResizedIm(np.hstack([img,erosion,canvas,]),"mark",2)
cv2.waitKey(0)
cv2.destroyAllWindows()
#VSL
VSL_img,VSL = findVSL(edges,showVSL = 0)
ShowResizedIm(np.hstack([VSL_img,]),"mark",1)
cv2.waitKey(0)
cv2.destroyAllWindows()
#VCC
xVCC,yVCC = findCoordinateMaxVCC(erosion,edges,VSL,semi_annula_radii)
previewAnnulaMask(erosion,xVCC,yVCC,semi_annula_radii)
#ODC final
x_out,y_out = findFinalODC(erosion,xVCC,yVCC,int(semi_annula_radii/2))
previewAnnulaMask(img,x_out,y_out,semi_annula_radii)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3, 4, 4, 4, 5, 5, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 8, 8, 9, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 7, 7, 7, 7, 7, 7, 7, 8, 8, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 8, 8, 9, 9, 8, 8, 9, 9, 9, 9, 9, 9, 9, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 6, 6, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 9, 9, 9, 9, 9, 10, 12, 11, 12, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5

In [70]:
#VCC
semi_annula_radii = 100
xVCC,yVCC = findCoordinateMaxVCC(erosion,edges,VSL,semi_annula_radii)
previewAnnulaMask(erosion,xVCC,yVCC,semi_annula_radii)
#ODC final
x_out,y_out = findFinalODC(erosion,xVCC,yVCC,int(semi_annula_radii/2))
previewAnnulaMask(img,x_out,y_out,semi_annula_radii)